# Part 3: Data Analytics

In [11]:
import pandas as pd
import json
import boto3
import io
from boto3.session import Session

### s3 Access functions

In [12]:
# Gloal vars
BUCKET = 'rearcdataquest'

def get_s3_session():
    AWS_KEY_ID = ''
    AWS_SECRET = ''
    session = Session(aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_SECRET)
    s3 = session.resource('s3')
    return s3

def read_s3_file(file_name):
    s3 = get_s3_session()
    obj = s3.Object(BUCKET, file_name)
    data = obj.get()['Body'].read()
    return data

### Load in dataframe and preprocess

In [13]:
# Load csv file Part1
data = read_s3_file('pr/pr.data.0.Current')
df1 = pd.read_csv(io.BytesIO(data), sep="\t")
df1.columns = ['series_id', 'year', 'period', 'value', 'footnote_codes']
df1 = df1.astype({"series_id": str, "period": str})
df1['series_id'] = df1['series_id'].str.strip()
df1['period'] = df1['period'].str.strip()
print(df1.columns)
print(df1.dtypes)

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')
series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object


In [14]:
df1

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
32404,PRS88003203,2020,Q03,110.307,NaN
32405,PRS88003203,2020,Q04,112.010,NaN
32406,PRS88003203,2020,Q05,112.635,NaN
32407,PRS88003203,2021,Q01,112.442,NaN


In [15]:
# Load json file Part2
data = read_s3_file('population_data.json')
df2 = pd.read_json(data)
print(df2.columns)
print(df2.dtypes)

Index(['ID Nation', 'Nation', 'ID Year', 'Year', 'Population', 'Slug Nation'], dtype='object')
ID Nation      object
Nation         object
ID Year         int64
Year            int64
Population      int64
Slug Nation    object
dtype: object


In [16]:
df2

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2019,2019,328239523,united-states
1,01000US,United States,2018,2018,327167439,united-states
2,01000US,United States,2017,2017,325719178,united-states
3,01000US,United States,2016,2016,323127515,united-states
4,01000US,United States,2015,2015,321418821,united-states
5,01000US,United States,2014,2014,318857056,united-states
6,01000US,United States,2013,2013,316128839,united-states


### Mean and std on DF2

In [17]:
'''
generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.
'''
q = df2[(df2["ID Year"] >= 2013) & (df2["ID Year"] <= 2018)]
mean = q['Population'].mean()
std = q['Population'].std()
print(f'mean = {mean}, std = {std}')

mean = 322069808.0, std = 4158441.040908095


### Best year report on DF1

In [18]:
'''
Using the dataframe from the time-series (Part 1), For every series_id, find the best year: 
the year with the max/largest sum of "value" for all quarters in that year. 
Generate a report with each series id, the best year for that series, and the summed value for that year. 
'''
q = df1.groupby(['series_id','year']).sum()
q = q.iloc[q.reset_index().groupby(['series_id'])['value'].idxmax()]

In [19]:
q

,,value
series_id,year,
PRS30006011,2012,9.500
PRS30006012,2014,8.800
PRS30006013,1998,733.290
PRS30006021,2010,14.200
PRS30006022,2010,11.200
...,...,...
PRS88003192,2002,285.200
PRS88003193,2014,520.840
PRS88003201,2007,21.700


### Join DFs on condition

In [20]:
'''
Using both dataframes from Part 1 and Part 2, 
generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and 
the population for that given year (if available in the population dataset)
'''
q = df1[(df1["series_id"] == 'PRS30006032') & (df1["period"] == 'Q01')]
j = q.merge(df2, left_on=['year'], right_on=['ID Year'])[['series_id','year','period','value','Population']]

In [21]:
j

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,2.2,316128839
1,PRS30006032,2014,Q01,-1.6,318857056
2,PRS30006032,2015,Q01,-1.2,321418821
3,PRS30006032,2016,Q01,-0.6,323127515
4,PRS30006032,2017,Q01,1.5,325719178
5,PRS30006032,2018,Q01,1.9,327167439
6,PRS30006032,2019,Q01,-3.1,328239523
